In [51]:
# @title INSTALAÇÃO DOS MÓDULOS NECESSÁRIOS

# Remver comentário da linha abaixo, caso seja necessário instalar o Pandas
#!pip install pandas -q

In [52]:
# @title IMPORTAÇÃO DAS BIBLIOTECAS

import pandas as pd
import sqlite3 as sq

In [53]:
# @title FUNÇÃO PARA VERIFICAR DADOS DE DATAFRAMES

def verficarDF(data_frame):
    n_linhas, n_colunas = data_frame.shape
    print(f"Número de Linhas: {n_linhas}")
    print(f"Número de Colunas: {n_colunas}")
    print("COLUNAS:")
    for index, coluna in enumerate(data_frame.columns):
        print(f"{index} - {coluna}")

In [54]:
# @title IMPORTAÇÃO DE DADOS DO ARQUIVO 'datatran.csv'

df = pd.read_csv("recursos/datatran2022.csv",sep=";",encoding="cp1252")
verficarDF(df)

Número de Linhas: 64547
Número de Colunas: 30
COLUNAS:
0 - id
1 - data_inversa
2 - dia_semana
3 - horario
4 - uf
5 - br
6 - km
7 - municipio
8 - causa_acidente
9 - tipo_acidente
10 - classificacao_acidente
11 - fase_dia
12 - sentido_via
13 - condicao_metereologica
14 - tipo_pista
15 - tracado_via
16 - uso_solo
17 - pessoas
18 - mortos
19 - feridos_leves
20 - feridos_graves
21 - ilesos
22 - ignorados
23 - feridos
24 - veiculos
25 - latitude
26 - longitude
27 - regional
28 - delegacia
29 - uop


In [55]:
# @title LIMPEZA DE DADOS

COLUNAS_EM_USO = ["km",
                  "fase_dia",
                  "sentido_via",
                  "tipo_pista",
                  "tracado_via",
                  "uso_solo",
                  "latitude",
                  "longitude",
                  "regional",
                  "delegacia",
                  "uop"]
df = df.drop(columns=COLUNAS_EM_USO)
verficarDF(df)

Número de Linhas: 64547
Número de Colunas: 19
COLUNAS:
0 - id
1 - data_inversa
2 - dia_semana
3 - horario
4 - uf
5 - br
6 - municipio
7 - causa_acidente
8 - tipo_acidente
9 - classificacao_acidente
10 - condicao_metereologica
11 - pessoas
12 - mortos
13 - feridos_leves
14 - feridos_graves
15 - ilesos
16 - ignorados
17 - feridos
18 - veiculos


In [56]:
# @title VERIFICAÇÃO DE INFORMAÇÕES NULAS OU DUPLICADAS

nulos = df.isnull().sum()
duplicados = df[df.duplicated()].sum()

n_linhas, _ = df.shape
nulos_pct = (nulos / n_linhas) * 100
duplicados_pct = (duplicados / n_linhas)  * 100

display(pd.DataFrame([nulos, nulos_pct, duplicados, duplicados_pct])
        .T
        .rename(columns={0:"Nulos", 1: "Nulos (%)", 2: "Duplicados", 3: "Duplicados (%)"})
        .round(decimals = 2)
       )

,Nulos,Nulos (%),Duplicados,Duplicados (%)
id,0.0,0.00,0.0,0.0
data_inversa,0.0,0.00,0.0,0.0
dia_semana,0.0,0.00,0.0,0.0
horario,0.0,0.00,0.0,0.0
uf,0.0,0.00,0.0,0.0
br,197.0,0.31,0.0,0.0
municipio,0.0,0.00,0.0,0.0
causa_acidente,0.0,0.00,0.0,0.0
tipo_acidente,0.0,0.00,0.0,0.0
classificacao_acidente,0.0,0.00,0.0,0.0


In [57]:
# @title MANIPULAÇÃO DA COLUNA 'br'

df['br'] = df['br'].fillna(0)
df['br'] = df['br'].astype('int64')

In [58]:
# @title CRIAÇÃO DE UMA BASE DE DADOS SQLITE

conn = sq.connect(f"datatran.db", check_same_thread=False)

In [59]:
# @title FUNÇÃO PARA ADCIONAR INFORMAÇÕES NO BANCO

def gravarInfo(query, conexao):
  try:
    cursor = conexao.cursor()
    cursor.execute(query)
    conexao.commit()
    cursor.close()
    print("Gravado com sucesso!")
    
  except Exception as e:
    print(f"Erro ao manipular BD: {e}")

In [60]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 1/4

# Renomeia as colunas
DICT_TABELAS_AUX = {'causa_acidente' : 'causas',
                    'tipo_acidente' : 'tipos',
                    'classificacao_acidente' : 'classificacoes',
                    'condicao_metereologica' : 'condicoes'}

lista_tabelas = []
for coluna_df, tabela_db in DICT_TABELAS_AUX.items():
  tabela = list(set(df[coluna_df]))
  tabela.sort()
  lista_tabelas.append({tabela_db : tabela})


In [61]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 2/4

# Função para criar tabelas dinamicamente
def gerarDLLAux(nome_tabela, caracteres):
  tipoDado = f"VARCHAR({caracteres})"
  query = f"""CREATE TABLE {nome_tabela} (cod SMALLINT PRIMARY KEY, descricao {tipoDado});
"""
  gravarInfo(query,conn)

In [62]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 3/4

# Função para inserir dados nas tabelas dinamicamente
map_tabelas = {}
def gerarDMLAux(nome_tabela, descricoes_tabela):
  for cod, descricao in enumerate(descricoes_tabela, 1):
    query = f"""INSERT INTO {nome_tabela} (cod, descricao) VALUES ('{cod}', '{descricao}');
"""
    gravarInfo(query,conn)
    map_tabelas[descricao] = cod

In [63]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 4/4

# Verificação do tamanho máximo de caracteres para tipos VARCHAr
for dictTabela in lista_tabelas:
    caracteres = max([len(x) for x in dictTabela.values()]) + 1

    # Chamada das funções
    for nome, descricoes in dictTabela.items():
        gerarDLLAux(nome,caracteres)
        gerarDMLAux(nome,descricoes)

Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com s

In [64]:
# @title GERAÇÃO DA TABELA DE REGIÕES 1/2

# Criação da tabela
query = f"""CREATE TABLE regioes (cod SMALLINT PRIMARY KEY, regiao VARCHAR(13));
"""
gravarInfo(query,conn)

Gravado com sucesso!


In [65]:
# @title GERAÇÃO DA TABELA DE REGIÕES 2/2

# Inserção de dados
dfRegioes = pd.read_csv("recursos/regioes.csv")
dados = zip(dfRegioes['cod'],dfRegioes['regiao'])
for cod, regiao in dados:
  query = f"""INSERT INTO regioes (cod, regiao) VALUES ('{cod}', '{regiao}');
"""
  gravarInfo(query,conn)

Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!


In [66]:
# @title GERAÇÃO DA TABELA DE ESTADOS 1/2

# Criação da tabela
query = f"""CREATE TABLE estados (
    cod SMALLINT PRIMARY KEY,
    uf VARCHAR(2),
    estado VARCHAR(20),
    cod_regiao SMALLINT,
    
    FOREIGN KEY (cod_regiao) REFERENCES regioes (cod));
"""
gravarInfo(query,conn)

Gravado com sucesso!


In [67]:
# @title GERAÇÃO DA TABELA DE ESTADOS 2/2

# Inserção de dados
map_estados = {}
dfEstados = pd.read_csv("recursos/estados.csv")
dados = zip(dfEstados['estado'], dfEstados['uf'], dfEstados['codigo'], dfEstados['regiao'])
for estado, uf, cod, cod_regiao in dados:
  query = f"""INSERT INTO estados (cod, uf, estado, cod_regiao) VALUES ('{cod}', '{uf}', '{estado}', '{cod_regiao}');
"""
  gravarInfo(query,conn)
  map_estados[uf] = cod

Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!


In [68]:
# @title GERAÇÃO DA TABELA DE MUNICIPIOS 1/2

# Criação da tabela
query = f"""CREATE TABLE municipios(
    cod SMALLINT PRIMARY KEY,
    municipio VARCHAR(33),
    cod_estado SMALLINT,
    
    FOREIGN KEY (cod_estado) REFERENCES estados (cod));
"""
gravarInfo(query,conn)

Gravado com sucesso!


In [69]:
# @title GERAÇÃO DA TABELA DE MUNICIPIOS 2/2

# Inserção de dados
dfMunicipios = df[['municipio','uf']]
dfMunicipios = dfMunicipios.drop_duplicates()
dados = zip(dfMunicipios['municipio'], dfMunicipios['uf'])

map_municipios = {}
cod = 1
for municipio, cod_estado in dados:
  query = f"""INSERT INTO municipios (cod, municipio, cod_estado) VALUES ('{cod}', '{municipio}', '{cod_estado}');
"""
  gravarInfo(query,conn)
  map_municipios[municipio] = cod
  cod +=1

Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com s

In [70]:
# @title GERAÇÃO DA TABELA DE DIAS DA SEMANA 1/2

# Criação da tabela
query = f"""CREATE TABLE dias_semana (cod SMALLINT PRIMARY KEY, dia_semana VARCHAR(11));
"""
gravarInfo(query,conn)


Gravado com sucesso!


In [71]:
# @title GERAÇÃO DA TABELA DE DIAS DA SEMANA 2/2

# Inserção de dados
map_dias = {}
dfDias = pd.read_csv("recursos/dias_semana.csv")
dados = zip(dfDias['cod'],dfDias['dia_semana'])
for cod, dia in dados:
  query = f"""INSERT INTO dias_semana (cod, dia_semana) VALUES ('{cod}', '{dia}');
"""
  gravarInfo(query,conn)
  map_dias[dia] = cod

Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!
Gravado com sucesso!


In [72]:
# @title GERAÇÃO DA TABELA DE REGISTROS DE ACIDENTES 1/2

# Criação da tabela 
query = f"""CREATE TABLE acidentes (
    cod INT PRIMARY KEY,
    data_acidente DATE,
    hora TIME,
    rodovia SMALLINT,
    mortos SMALLINT,
    feridos_leves SMALLINT,
    feridos_graves SMALLINT,
    ilesos SMALLINT,
    ignorados SMALLINT,
    veiculos SMALLINT);
"""
gravarInfo(query,conn)

Gravado com sucesso!


In [74]:
# @title GERAÇÃO DA TABELA DE REGISTROS DE ACIDENTES 2/2

# Renomeia as colunas
DICT_ACID = {'id' : 'cod',
            'data_inversa' : 'data_acidente',
            'horario' : 'hora',
            'br' : 'rodovia',
            'mortos' : 'mortos',
            'feridos_leves' : 'feridos_leves',
            'feridos_graves' : 'feridos_graves',
            'ilesos' : 'ilesos',
            'ignorados' : 'ignorados',
            'veiculos' : 'veiculos'}

# Tratamento de informações
acidentes = df[DICT_ACID.keys()]
acidentes = acidentes.rename(columns= DICT_ACID)

# Inserção de dados
acidentes.to_sql('acidentes', con= conn, if_exists= 'append', index=False)

64547

In [75]:
# @title GERAÇÃO DA TABELA DE OCORRÊNCIA 1/2

# criação da tabela
query = f"""CREATE TABLE ocorrencias (
  cod_acidente INT,
  cod_dia SMALLINT,
  cod_estado SMALLINT,
  cod_municipio SMALLINT,
  cod_causa SMALLINT,
  cod_tipo SMALLINT,
  cod_classificacao SMALLINT,
  cod_condicao SMALLINT,
  
  FOREIGN KEY (cod_acidente) REFERENCES acidentes (cod),
  FOREIGN KEY (cod_dia) REFERENCES dias_semana (cod),
  FOREIGN KEY (cod_estado) REFERENCES estados (cod),
  FOREIGN KEY (cod_municipio) REFERENCES municipios (cod),
  FOREIGN KEY (cod_causa) REFERENCES causas (cod),
  FOREIGN KEY (cod_tipo) REFERENCES tipos (cod),
  FOREIGN KEY (cod_classificacao) REFERENCES classificacoes (cod),
  FOREIGN KEY (cod_condicao) REFERENCES condicoes (cod));
"""
gravarInfo(query,conn)

Gravado com sucesso!


In [77]:
# @title GERAÇÃO DA TABELA DE OCORRÊNCIA 2/2

# Renomeia as colunas
DICT_OCOR = {'id' : 'cod_acidente',
            'dia_semana' : 'cod_dia',
            'uf' : 'cod_estado',
            'municipio' : 'cod_municipio',
            'causa_acidente' : 'cod_causa',
            'tipo_acidente' : 'cod_tipo',
            'classificacao_acidente' : 'cod_classificacao',
            'condicao_metereologica' : 'cod_condicao'}

# Tratamento de informações
ocorrencias = df[DICT_OCOR.keys()]
for coluna in DICT_TABELAS_AUX.keys():
  ocorrencias[coluna] = ocorrencias[coluna].map(map_tabelas)

ocorrencias['uf'] = ocorrencias['uf'].map(map_estados)
ocorrencias['municipio'] = ocorrencias['municipio'].map(map_municipios)
ocorrencias['dia_semana'] = ocorrencias['dia_semana'].map(map_dias)
ocorrencias = ocorrencias.rename(columns= DICT_OCOR)

# Inserção de dados
ocorrencias.to_sql('ocorrencias', con= conn, if_exists= 'append', index=False)

/tmp/ipykernel_16055/1081984699.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocorrencias[coluna] = ocorrencias[coluna].map(map_tabelas)
/tmp/ipykernel_16055/1081984699.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocorrencias[coluna] = ocorrencias[coluna].map(map_tabelas)
/tmp/ipykernel_16055/1081984699.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

64547